<a href="https://colab.research.google.com/github/newning117/Blog-Crawling/blob/main/github_blog_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ** 00 블로그 수치_2025년 00월 00일 00시 00분 기준**

<흐름 정리>

**A. Colab에서 크롤링 실행 및 DataFrame 생성**
    
    1. url
    2. date
    3. title
    4. comment_count
    5. like_count

**B. 1–5번 수치 산출**

    1. 총 게시물 수
    2. 총 댓글 수
    3. 총 좋아요 수
    4. 게시물 1건 당 평균 댓글 수
    5. 게시물 1건 당 평균 좋아요 수

**C\. 추가 기초 지표**

    1. 	댓글/공감 분포 (중앙값, 분위수, 0개 비율)
	2.	시간축(연도/월별, 최근 6~12개월 기준) 지표
	3.	공감/댓글 TOP N / LOW N 리스트
	4.	태그나 주제별 요약 (가능해지면)

# **A. Colab에서 크롤링 실행**

##A-1. 라이브러리 설치

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install beautifulsoup4 gspread


In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

## A-2. list_page*.html 에서 logNo만 뽑기 (JSON/HTML 둘 다 대응)

In [ ]:
import os
import requests, json, re, html as html_lib

BASE_BLOG = "https://blog.naver.com"
BLOG_ID = #"👉⭐️해당 블로그 아이디 기재"

HEADERS = {
    #"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 "
                  #"(KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
} #👉⭐️사용자의 User-Agent 기재

# ✅ 저장할 폴더 (구글 드라이브)
SAVE_DIR = #"👉⭐️/content/drive 저장 경로 경로 기재 "
os.makedirs(SAVE_DIR, exist_ok=True)
print("저장 경로:", SAVE_DIR)

post_lognos = set()

# 1~12페이지 정도면 299개 글 커버 (필요하면 range 끝 숫자 늘리면 됨)
for page in range(1, 20): #👉⭐️목록 리스트의 페이지 개수를 1~20페이지까지 설정한다는 의미로 필요에 따라 20을 다른 수로 변경
    url = (
        "https://blog.naver.com/PostTitleListAsync.naver"
        f"?blogId={BLOG_ID}&viewdate=&currentPage={page}"
        "&categoryNo=0&parentCategoryNo=&countPerPage=30" #PerPage=30 -> 1페이지당 30개 목록을 의미
    )
    print(f"\n[요청] page={page} ->", url)
    res = requests.get(url, headers=HEADERS)
    print("  status:", res.status_code)

    if res.status_code != 200:
        print("  ⚠️ 200이 아니라서 이 페이지는 스킵:", res.text[:80])
        continue

    text = res.text.strip()

    # ✅ 1) 받은 응답을 그대로 html 파일로 저장
    save_path = os.path.join(SAVE_DIR, f"list_page{page}.html")
    with open(save_path, "w", encoding="utf-8") as f:
        f.write(text)
    print("  👉 파일 저장 완료:", save_path)

    # ✅ 2) logNo 추출 로직
    if text.startswith("{"):
        # JSON 응답일 가능성
        try:
            data = json.loads(text)
        except Exception as e:
            print("  JSON 파싱 실패, 문자열로 처리:", e)
            fragment = text
        else:
            if (data.get("isSuccess") is False) and ("error" in data):
                print("  ⚠️ API 에러 응답, 이 페이지는 logNo 없음 (message:",
                      data["error"].get("message"), ")")
                continue

            found_here = 0

            # (1) postList 안의 logNo
            if isinstance(data.get("postList"), list):
                for post in data["postList"]:
                    ln = str(post.get("logNo", "")).strip()
                    if ln.isdigit():
                        post_lognos.add(ln)
                        found_here += 1

            # (2) 혹시 몰라 문자열 전체에서 정규식으로 한 번 더
            fragment_parts = []
            for v in data.values():
                if isinstance(v, str):
                    fragment_parts.append(v)
            fragment = html_lib.unescape("".join(fragment_parts))

            found_eq  = re.findall(r"logNo=(\d+)", fragment)
            found_key = re.findall(r'"logNo"\s*:\s*(\d+)', fragment)
            for ln in found_eq + found_key:
                post_lognos.add(ln)
                found_here += 1

            print("  이 페이지에서 찾은 logNo 개수:", found_here)
            continue

    # JSON 아닐 때 (그냥 문자열에서 logNo 패턴 찾기)
    fragment = html_lib.unescape(text)
    found_eq  = re.findall(r"logNo=(\d+)", fragment)
    found_key = re.findall(r'"logNo"\s*:\s*(\d+)', fragment)
    found = found_eq + found_key

    print("  이 페이지에서 찾은 logNo 개수:", len(found))

    for ln in found:
        post_lognos.add(ln)

# ✅ 최종 정리
print("\n✅ 최종 수집된 logNo 개수(중복 제거 후):", len(post_lognos))

post_urls = [
    f"{BASE_BLOG}/PostView.naver?blogId={BLOG_ID}&logNo={ln}"
    for ln in sorted(post_lognos)
]

print("예시 URL 5개:")
for u in post_urls[:5]:
    print(" ", u)

##A-3.  각 게시물 상세 페이지에서 정보 수집 (제목/날짜/댓글/좋아요/태그)

###A-3-1. mainFrame까지 들어가는 함수

In [ ]:
#url로 제목 가져오기

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
} #👉⭐️사용자의 User-Agent 기재

def fetch_soup_following_mainframe(url: str) -> BeautifulSoup:
    # 상대경로면 절대경로로
    if url.startswith("/"):
        full_url = BASE_BLOG + url
    else:
        full_url = url

    res = requests.get(full_url, headers=HEADERS)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")

    # 프레임셋 → mainFrame 안으로 한 번 더 들어가기
    iframe = soup.find("iframe", id="mainFrame")
    if iframe and iframe.get("src"):
        src = iframe["src"]
        if src.startswith("/"):
            src = BASE_BLOG + src
        res2 = requests.get(src, headers=HEADERS)
        res2.raise_for_status()
        soup = BeautifulSoup(res2.text, "html.parser")

    return soup


In [ ]:
#확인 코드
soup = fetch_soup_following_mainframe(#"확인용 포스트 url 기입")
print(soup.title.string)

###A-3-2. 상세 정보 파싱 함수(제목, 업로드 일자, 댓글 수)

In [ ]:
# mainframe에서 url, 제목, 업로드 일자, 댓글 수 가져오기
def parse_post_detail(url: str) -> dict:
    soup = fetch_soup_following_mainframe(url)


    # 1) 제목
        title = None
    og = soup.find("meta", property="og:title")
    if og and og.get("content"):
        title = og["content"].strip()
    if not title:
        t_el = (
            soup.select_one(".pcol1.itemSubjectBoldfont") or
            soup.select_one("h3.se_textarea") or
            soup.select_one("h2")
        )
        if t_el:
            title = t_el.get_text(strip=True)


    # 2) 업로드 날짜
        date = None
    d_el = (
        soup.select_one(".se_publishDate") or
        soup.select_one(".se_publishDate.pcol2") or
        soup.select_one("span.t_date")
    )
    if d_el:
        date = d_el.get_text(strip=True)

    full_text = soup.get_text(" ", strip=True)


    # 3) 댓글 수
        comment_count = 0

    cmt_el = soup.select_one(".u_cbox_count, .u_cbox_cnt")
    if cmt_el:
        m = re.search(r"\d+", cmt_el.get_text())
        if m:
            comment_count = int(m.group())

    if comment_count == 0:
        m = re.search(r"댓글\s*([0-9][0-9,]*)", full_text)
        if m:
            comment_count = int(m.group(1).replace(",", ""))


    return {
        "url": url,
        "title": title,
        "date": date,
        "comment_count": comment_count
    }

In [ ]:
#정상 실행 확인
print(parse_post_detail(#"확인용 포스트 url 기입"))

###A-3-3. 상세 정보 파싱 함수(공감 수)

In [ ]:
#공감 수 파싱 위해서 HTML 내 span 함수 찾기
test_url = #"확인용 포스트 url 기입"

soup = fetch_soup_following_mainframe(test_url)

# 1) 우리가 원하는 span이 몇 개 있는지
spans = soup.select("span.u_likeit_text._count.num")
print("찾은 공감 span 개수:", len(spans))

for i, el in enumerate(spans):
    print(f"[{i}] span HTML:", el)
    print(f"[{i}] span text:", el.get_text(strip=True))

# 2) 혹시라도 문자열로만 있는지
print("'u_likeit_text _count num' in 전체 HTML? ->", "u_likeit_text _count num" in str(soup))

In [ ]:
# JS로 공감 리스트 정상 파싱하기
import requests, time, json


HEADERS_API = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
    "Accept": "*/*",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Content-Type": "application/x-www-form-urlencoded; charset=utf-8",
    "Cookie": "NNB=...; NID_SES=...; JSESSIONID=...", #👉⭐️사용자의 Cookie 기재
    "Referer": "https://blog.naver.com/SympathyHistoryList.naver?blogId=...&logNo=...&layoutWidthClassName=...", #👉⭐️사용자의 Referer 기재
}

def extract_logno(url: str):
    import re
    m = re.search(r"logNo=(\d+)", url)
    return m.group(1) if m else None

test_url = #"테스트 포스팅 url 기입"
logno = extract_logno(test_url)

ts = int(time.time() * 1000)
api_url = f"{BASE_BLOG}/api/blogs/{BLOG_ID}/posts/{logno}/sympathy-users"
params = {"itemCount": 60, "timeStamp": ts}

res = requests.get(api_url, headers=HEADERS_API, params=params)
print("status:", res.status_code)

data = res.json()
print(json.dumps(data, ensure_ascii=False, indent=2)[:1200])

In [ ]:
#TEST URL 통해 공감 수 정상 파싱 확인하기
def get_like_count_via_api(url: str) -> int | None:
    logno = extract_logno(url)
    if not logno:
        print("logNo 추출 실패:", url)
        return None

    ts = int(time.time() * 1000)
    api_url = f"{BASE_BLOG}/api/blogs/{BLOG_ID}/posts/{logno}/sympathy-users"
    params = {"itemCount": 1, "timeStamp": ts}

    res = requests.get(api_url, headers=HEADERS_API, params=params)
    if res.status_code != 200:
        print("API 에러:", res.status_code, res.text[:200])
        return None

    data = res.json()
    result = data.get("result")
    if not isinstance(result, dict):
        print("result 구조 이상:", type(result))
        return None

    # 1) 가장 흔한 케이스: totalCount
    if "totalCount" in result:
        try:
            return int(result["totalCount"])
        except Exception:
            pass

    # 2) 다른 이름일 수도 있음: sympathyUsersCount, sympathyCount, likeCount 등
    for key in ["sympathyUsersCount", "sympathyCount", "likeCount"]:
        if key in result:
            try:
                return int(result[key])
            except Exception:
                pass

    # 3) sympathyUsers 배열 길이로 계산
    if "sympathyUsers" in result and isinstance(result["sympathyUsers"], list):
        return len(result["sympathyUsers"])

    print("공감 수 키를 찾지 못했어요. result keys:", list(result.keys()))
    return None

In [ ]:
#랜덤 url로 확인
print(get_like_count_via_api(test_url))

In [ ]:
# 전체 게시물 공감 수 파싱하기
rows = []

for url in post_urls:
    try:
        like_cnt = get_like_count_via_api(url)
    except Exception as e:
        print("에러 발생:", url, "/", e)
        like_cnt = None

    rows.append({
        "url": url,
        "like_count": like_cnt,
    })

df_likes = pd.DataFrame(rows)
df_likes.head()

##A-4. 전체 게시물 DataFrame 만들기

### df_detail 생성(url, title, date, comment_count)

In [ ]:
# 1. df_detail 생성
rows_detail = []

for url in post_urls:
    try:
        info = parse_post_detail(url)   # url, title, date, comment_count 리턴
    except Exception as e:
        print("에러 발생:", url, "/", e)
        continue
    rows_detail.append(info)

df_detail = pd.DataFrame(rows_detail)
df_detail.head()

### df_merged 생성 (df_detail + like_count)

In [ ]:
#2. df_likes 생성 및 df_detail과 merge
df_merged = df_detail.merge(df_likes, on="url", how="left")
df_merged.head()

#**B. 1-5번 수치 산출**

    1. 총 게시물 수
    2. 총 댓글 수
    3. 총 좋아요 수
    4. 게시물 1건 당 평균 댓글 수
    5. 게시물 1건 당 평균 좋아요 수

In [ ]:
df_merged.info()

In [ ]:
# NaN → 0으로 바꿔두는 게 깔끔
df_merged["comment_count"] = df_merged["comment_count"].fillna(0).astype(int)
df_merged["like_count"]    = df_merged["like_count"].fillna(0).astype(int)

# 5. 총 게시물 수
total_posts = len(df_merged)

# 6. 총 댓글 수
total_comments = df_merged["comment_count"].sum()

# 7. 총 좋아요 수
total_likes = df_merged["like_count"].sum()

# 8. 게시물 1건 당 평균 댓글 수
avg_comments_per_post = total_comments / total_posts if total_posts > 0 else np.nan

# 9. 게시물 1건 당 평균 좋아요 수
avg_likes_per_post = total_likes / total_posts if total_posts > 0 else np.nan

print("총 게시물 수:", total_posts)
print("총 댓글 수:", total_comments)
print("총 좋아요 수:", total_likes)
print("게시물 1건 당 평균 댓글 수:", avg_comments_per_post)
print("게시물 1건 당 평균 좋아요 수:", avg_likes_per_post)

#**C. 추가 기초 지표**

    1. 	댓글/공감 분포 (중앙값, 분위수, 0개 비율)
	2.	시간축(연도/월별, 최근 6~12개월 기준) 지표


###C-1. 댓글/공감 분포 (중앙값, 분위수, 0개 비율)

In [ ]:
# 0) 안전하게 타입/결측치 처리
df_merged["comment_count"] = df_merged["comment_count"].fillna(0).astype(int)
df_merged["like_count"]    = df_merged["like_count"].fillna(0).astype(int)

# 1) 댓글 분포 요약 (댓글 수)
comment_stats = df_merged["comment_count"].describe(percentiles=[0.25, 0.5, 0.75, 0.9])
print("📌 댓글 분포 요약")
print(comment_stats)
print()

# 2) 공감(좋아요) 분포 요약 (like_count)
like_stats = df_merged["like_count"].describe(percentiles=[0.25, 0.5, 0.75, 0.9])
print("📌 공감(좋아요) 분포 요약")
print(like_stats)
print()

# 3) 댓글/공감 0개인 글 비율
zero_comment_ratio = (df_merged["comment_count"] == 0).mean()
zero_like_ratio    = (df_merged["like_count"] == 0).mean()

print(f"💬 댓글 0개인 글 비율: {zero_comment_ratio:.2%}")
print(f"❤️ 공감 0개인 글 비율: {zero_like_ratio:.2%}")

# 4) 상위 10% 기준 임계값도 보고 싶다면 (선택)
comment_top10_threshold = df_merged["comment_count"].quantile(0.9)
like_top10_threshold    = df_merged["like_count"].quantile(0.9)

print()
print(f"💬 댓글 수 상위 10% 기준 임계값(대략): {comment_top10_threshold}")
print(f"❤️ 공감 수 상위 10% 기준 임계값(대략): {like_top10_threshold}")

###C-2. 시간축(연도/월별, 최근 6~12개월 기준) 지표

* 날짜를 datetime으로 변환하기

In [ ]:
# 혹시 모를 결측·타입 처리
df_merged["comment_count"] = df_merged["comment_count"].fillna(0).astype(int)
df_merged["like_count"]    = df_merged["like_count"].fillna(0).astype(int)

# 'date' 예시: "2023. 5. 14. 17:31"
# 문자열 → datetime
df_merged["dt"] = pd.to_datetime(df_merged["date"], errors="coerce")

# 날짜 파싱 안 된 행은 잠깐 버리자 (극소수일 가능성 큼)
df_time = df_merged.dropna(subset=["dt"]).copy()

* 연도별 지표
    * 연도별 게시물 수
    * 연도별 평균 댓글 수 / 평균 공감 수

In [ ]:
df_time["year"] = df_time["dt"].dt.year

yearly_stats = (
    df_time
    .groupby("year")
    .agg(
        posts=("url", "count"),
        avg_comments=("comment_count", "mean"),
        avg_likes=("like_count", "mean"),
    )
    .reset_index()
)

print("📆 연도별 통계")
print(yearly_stats)

* 연-월별 지표 (월 단위 추이)
 	* 월별 게시물 수
	* 월별 평균 댓글 / 공감 수

In [ ]:
# 연-월 문자열 컬럼 (예: 2023-05)
df_time["year_month"] = df_time["dt"].dt.to_period("M").astype(str)

monthly_stats = (
    df_time
    .groupby("year_month")
    .agg(
        posts=("url", "count"),
        avg_comments=("comment_count", "mean"),
        avg_likes=("like_count", "mean"),
    )
    .reset_index()
    .sort_values("year_month")
)

print("📆 월별 통계 (연-월 단위)")
print(monthly_stats.tail)  # 최근 12개월만 확인해보기

* 최근 6개월 / 12개월 분석

In [ ]:
latest_date = df_time["dt"].max()
cut_6m  = latest_date - pd.DateOffset(months=6)
cut_12m = latest_date - pd.DateOffset(months=12)

recent_6m  = df_time[df_time["dt"] >= cut_6m]
recent_12m = df_time[df_time["dt"] >= cut_12m]

def summarize_period(name, df):
    n_posts = len(df)
    total_comments = df["comment_count"].sum()
    total_likes    = df["like_count"].sum()
    avg_c = total_comments / n_posts if n_posts > 0 else np.nan
    avg_l = total_likes / n_posts if n_posts > 0 else np.nan

    print(f"🔹 {name}")
    print("  기간:", df["dt"].min(), "~", df["dt"].max())
    print("  게시물 수:", n_posts)
    print("  총 댓글 수:", total_comments)
    print("  총 공감 수:", total_likes)
    print("  글당 평균 댓글 수:", avg_c)
    print("  글당 평균 공감 수:", avg_l)
    print()

summarize_period("최근 6개월", recent_6m)
summarize_period("최근 12개월", recent_12m)